# Połączenie z bazą danych i podstawowa obsługa

In [103]:
from pymongo import MongoClient

def create_connection(db_url):
    client = MongoClient(db_url) # tworzę połączenie z baza danych
    return client,client['builder_Konrad'] #tworzę indiwidualnaą baZe danych z jej nazwą


db_url = "mongodb://localhost:27017" # daje link do bazy danych - lokalnej
client,db = create_connection(db_url) #tworzę połączenie z bazą danych

In [104]:
def create_collection(db,collection_name):
    return db[collection_name]

cities = create_collection(db,'cities')
buildings = create_collection(db,'buildings')

# dodaje jedno miasto

In [105]:
def create_city(collection,city):
    result = collection.insert_one(city) # dodaje jeden fragment w formacie słownika
    return result.inserted_id # zwracam ID dla utowrzonego miata


city_id = create_city(cities,{'name':"Poznan",'latidude':52.2,'longitude':17.0})
print(f'ID nowego miasta {city_id}') # wyświetla mID nowego miasta

#przejrzę miasta w bazie
# zapiszę ID wstawionego miasta 

ID nowego miasta 6651fb112412fd105dbb6d92


In [106]:
cursor = cities.find()

for doc in cursor:
    print(doc)

poznan = cities.find_one({'name':'Poznan'}) # znajduje baze danych po nazwie
poznan_id = poznan['_id'] # tak wyswietlam dane id
print('Poznan ma ID:',poznan_id)

{'_id': ObjectId('6651fae62412fd105dbb6d8e'), 'name': 'Poznan', 'latidude': 52.2, 'longitude': 17.0}
{'_id': ObjectId('6651fb112412fd105dbb6d92'), 'name': 'Poznan', 'latidude': 52.2, 'longitude': 17.0}
Poznan ma ID: 6651fae62412fd105dbb6d8e


In [107]:
# wstawię dwa budynki do tego samego miasta

def create_building(collection,building):
    result = collection.insert_one(building)
    return result.inserted_id

buildings_1 = create_building(buildings,{'city_id':poznan_id,"name":"Galeria","height":17.0,'build_year':2010})
buildings_2 = create_building(buildings,{'city_id':poznan_id,"name":"Teatr","height":25.5,'build_year':1950})

In [108]:
print("wstawiono Galerię o ID: ", buildings_1)
print("Wstawiono Teatr o ID: ", buildings_2)

wstawiono Galerię o ID:  6651fb112412fd105dbb6d93
Wstawiono Teatr o ID:  6651fb112412fd105dbb6d94


In [109]:
# wyswietla wszystkie budynki w bazie

cursor = buildings.find()

for doc in cursor:
    print(doc)

{'_id': ObjectId('6651f9262412fd105dbb6d7b'), 'city_id': ObjectId('6651f9262412fd105dbb6d7a'), 'name': 'Galeria', 'height': 17.0, 'build_year': 2010}
{'_id': ObjectId('6651f92c2412fd105dbb6d7f'), 'city_id': ObjectId('6651f9262412fd105dbb6d7a'), 'name': 'Galeria', 'height': 17.0, 'build_year': 2010}
{'_id': ObjectId('6651fae62412fd105dbb6d8f'), 'city_id': ObjectId('6651fae62412fd105dbb6d8e'), 'name': 'Galeria', 'height': 17.0, 'build_year': 2010}
{'_id': ObjectId('6651fb112412fd105dbb6d93'), 'city_id': ObjectId('6651fae62412fd105dbb6d8e'), 'name': 'Galeria', 'height': 17.0, 'build_year': 2010}
{'_id': ObjectId('6651fb112412fd105dbb6d94'), 'city_id': ObjectId('6651fae62412fd105dbb6d8e'), 'name': 'Teatr', 'height': 25.5, 'build_year': 1950}


In [110]:
# połącze dwie kolekcje pod każdym miatem znajdą się szczegóły miasta
result = buildings.aggregate([
    {
        "$lookup":{
            "from":"cites",
            "localField":"city_id",
            "foreignField":"_id",
            "as":"city"
        }
    }
])

for doc in result:
    print(doc)

{'_id': ObjectId('6651f9262412fd105dbb6d7b'), 'city_id': ObjectId('6651f9262412fd105dbb6d7a'), 'name': 'Galeria', 'height': 17.0, 'build_year': 2010, 'city': []}
{'_id': ObjectId('6651f92c2412fd105dbb6d7f'), 'city_id': ObjectId('6651f9262412fd105dbb6d7a'), 'name': 'Galeria', 'height': 17.0, 'build_year': 2010, 'city': []}
{'_id': ObjectId('6651fae62412fd105dbb6d8f'), 'city_id': ObjectId('6651fae62412fd105dbb6d8e'), 'name': 'Galeria', 'height': 17.0, 'build_year': 2010, 'city': []}
{'_id': ObjectId('6651fb112412fd105dbb6d93'), 'city_id': ObjectId('6651fae62412fd105dbb6d8e'), 'name': 'Galeria', 'height': 17.0, 'build_year': 2010, 'city': []}
{'_id': ObjectId('6651fb112412fd105dbb6d94'), 'city_id': ObjectId('6651fae62412fd105dbb6d8e'), 'name': 'Teatr', 'height': 25.5, 'build_year': 1950, 'city': []}


In [111]:
# zaaktualizuje wysokość budynku

def update_buildind_height(buildings,building_id,height):
    buildings.update_one({"id":building_id},{"$set":{"height":height}})

galeria = buildings.find_one({"name":"Galeria"})
update_buildind_height(buildings,galeria['_id'],25.4)

In [112]:
#usuń budynek

def delete_building(buildings,building_id):
    buildings.delete_one({"_id":building_id})

teatr = buildings.find_one({"name":"Teatr"})
delete_building(buildings,teatr["_id"])

#client.close()

# Serwer umożliwiający obsługę bazy danych MongoDB

In [113]:
import json
from fastapi import FastAPI
import uvicorn

app = FastAPI()
#dodam możliwość pobrania wszystkich miast
@app.get('/cities')
def cities_get():
    all_cities = cities.find()
    output = [city for city in all_cities]
    return output

# zamień
all_cities = cities.find()
# na 
all_cities = cities.find({}, {'_id': False})

# do kodu
all_cities = cities.find()
output = [city for city in all_cities]
# dodaj
for city in output:
    city['_id'] = str(city['_id'])


if __name__=='__main__':
    uvicorn.run(app)

#zamknij klienta
#client.close()

RuntimeError: asyncio.run() cannot be called from a running event loop